In [2]:
import torch
import numpy as np


def parse_data_stats(file_path):
    # Load the stats with weights_only=False
    stats = torch.load(file_path, weights_only=False)

    # Parse and print statistics
    print(f"Loading stats from: {file_path}")

    pose_mean = stats["pose_mean"].reshape(-1)
    pose_std = stats["pose_std"].reshape(-1)
    face_mean = stats["code_mean"].reshape(-1)
    face_std = stats["code_std"].reshape(-1)
    audio_mean = stats["audio_mean"].reshape(-1)
    audio_std = stats["audio_std_flat"].reshape(-1)
    print(stats.keys())
    print(f"Pose mean shape: {pose_mean.shape}")
    print(f"Pose std shape: {pose_std.shape}")
    print(f"Face mean shape: {face_mean.shape}")
    print(f"Face std shape: {face_std.shape}")
    print(f"Audio mean shape: {audio_mean}")
    print(f"Audio std shape: {audio_std}")

    return stats

# Load stats
stats = parse_data_stats("./dataset/GQS883/data_stats.pth")

Loading stats from: ./dataset/GQS883/data_stats.pth
dict_keys(['code_std_flat', 'code_std', 'code_mean', 'pose_std_flat', 'pose_std', 'pose_mean', 'audio_std_flat', 'audio_std', 'audio_mean'])
Pose mean shape: (104,)
Pose std shape: (104,)
Face mean shape: (256,)
Face std shape: (256,)
Audio mean shape: [-7.8997459e-08 -1.5775761e-07]
Audio std shape: [0.01367152]


In [1]:
import requests
import json
import os


def create_multipart_request():
    # Prepare the files
    files = []
    base_dir = './dataset/GQS883/'
    audio_files = ['scene02_audio.wav']

    for filename in audio_files:
        filepath = os.path.join(base_dir, filename)
        files.append(('file', (filename, open(filepath, 'rb'))))

    # Prepare the request
    response = requests.post(
        "https://api.hume.ai/v0/batch/jobs",
        headers={
            "X-Hume-Api-Key": "1sJm18G85hBifTpuITNw8OvubszP0rAdstFuf6EWfAP8I2a6"
        },
        data={
            'json': json.dumps({
                "models": {
                    "burst": {},
                    "prosody": {},
                    "language": {},
                    "ner": {}
                }
            }),
        },
        files=files
    )

    return parse_job_response(response)

def parse_job_response(response):
   try:
       return response.json().get('job_id')
   except ValueError:
       print("Error parsing JSON")
       return None
# Execute the request
response = create_multipart_request()

# Print response details
print("Status Code:", response)

Status Code: 998c2967-7398-428a-8073-17b2ec27b328


In [22]:
import requests

# Get job predictions (GET /v0/batch/jobs/:id/predictions)
response = requests.get(
  "https://api.hume.ai/v0/batch/jobs/aaa72c03-c345-467f-b076-296960463683/predictions",
  headers={
    "X-Hume-Api-Key": "1sJm18G85hBifTpuITNw8OvubszP0rAdstFuf6EWfAP8I2a6"
  },
)

print(json.dumps(response.json(), indent=2))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [3]:
def save_to_json(data, filename='response.json'):
   """保存数据到JSON文件"""
   with open(filename, 'w', encoding='utf-8') as f:
       json.dump(data, f, indent=2, ensure_ascii=False)
   print(f"Data saved to {filename}")

import json
import requests

# Get job predictions (GET /v0/batch/jobs/:id/predictions)
response = requests.get(
  "https://api.hume.ai/v0/batch/jobs/998c2967-7398-428a-8073-17b2ec27b328/predictions",
  headers={
    "X-Hume-Api-Key": "1sJm18G85hBifTpuITNw8OvubszP0rAdstFuf6EWfAP8I2a6"
  },
)

# 保存响应数据
save_to_json(response.json())

Data saved to response.json
